# 203Agent Openai Assistants
Assistants API 允许您在自己的应用程序中构建 AI 助手。

您可以使用 OpenAI 工具或自定义工具与 OpenAI 助手进行交互。当专门使用 OpenAI 工具时，您只需直接调用助手即可获得最终答案。使用自定义工具时，您可以使用内置的 AgentExecutor 运行助手和工具执行循环，也可以轻松编写自己的执行器。

In [9]:
#设置代理
import os

os.environ['http_proxy'] = 'http://127.0.0.1:10809'
os.environ['https_proxy'] = 'http://127.0.0.1:10809'

# 这里我们需要用到google的搜索api
# 在这个网址https://serpapi.com/search-api，但是需要你用邮箱验证和手机号验证
os.environ["SERPAPI_API_KEY"] = "5266d1ac4xxxxxxxxxxxxxxx91"

In [3]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",
    instructions="你是一名私人数学辅导员。编写并运行代码来回答数学问题。",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview",
)
output = interpreter_assistant.invoke({"content": "10 - 4 与 2.7 相比提高了多少？"})
output

[ThreadMessage(id='msg_fAMj9oaH1Tz7JtSVv2evKWpF', assistant_id='asst_rjIhBCEKyIYq5GKmg1UhMdxZ', content=[MessageContentText(text=Text(annotations=[], value='10 - 4 的结果是 6，与 2.7 相比提高了大约 122.22%。'), type='text')], created_at=1703828765, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_pmFLgqS6mGGVdDsRteayDWgY', thread_id='thread_0yIF1UlI0N4sZHh6uOH7xCzx')]

## 使用OpenAI functions
某些 OpenAI 模型（如 gpt-3.5-turbo-0613 和 gpt-4-0613）已经过微调，可以检测何时应该调用函数，并使用应传递给函数的输入进行响应。在 API 调用中，您可以描述函数，并让模型智能地选择输出包含参数的 JSON 对象来调用这些函数。OpenAI 函数 API 的目标是比通用文本完成或聊天 API 更可靠地返回有效且有用的函数调用。

In [7]:
! pip install openai google-search-results

## 初始化工具

In [11]:
! pip install numexpr

   ---------------------------------------- 0.0/96.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/96.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/96.4 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.4 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.4 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.4 kB ? eta -:--:--
   ------------ --------------------------- 30.7/96.4 kB 146.3 kB/s eta 0:00:01
   ---------------- ----------------------- 41.0/96.4 kB 140.9 kB/s eta 0:00:01
   ---------------- ----------------------- 41.0/96.4 kB 140.9 kB/s eta 0:00:01
   ---------------- ----------------------- 41.0/96.4 kB 140.9 kB/s eta 0:00:01
   ---------------- ----------------------- 41.0/96.4 kB 140.9 kB/s eta 0:00:01
   ---------------- ----------------------- 41.0/96.4 kB 140.9 kB/s eta 0:00:01
   ------------------------- -------------- 61.4/96.4 kB 109.4 kB/s eta 0:00:01
   --------------

In [12]:

from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper, SQLDatabase

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

In [13]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


In [14]:
from langchain.tools.render import format_tool_to_openai_function
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [15]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [16]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "谁是吴京的老婆？她现在的年龄是多少？"
    }
)



> Entering new AgentExecutor chain...

Invoking: `Search` with `吴京的老婆`


Xie Nan
Invoking: `Search` with `吴京的老婆谢楠年龄`


吴京谢楠年龄差是多少九岁可是有相当大的代沟了_平平... 2022年5月31日据两人资料显示,谢楠生于1983年,吴京生于1974年,这样算来,谢楠吴京相差9岁。吴京的妻子是谢楠，她的年龄是1983年出生的，目前大约是39岁。

> Finished chain.


{'input': '谁是吴京的老婆？她现在的年龄是多少？', 'output': '吴京的妻子是谢楠，她的年龄是1983年出生的，目前大约是39岁。'}

## 直接使用 OpenAIFunctionsAgent

In [17]:
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

In [18]:
agent_executor.invoke(
    {
        "input": "谁是吴京的老婆？她现在的年龄是多少？"
    }
)



> Entering new AgentExecutor chain...

Invoking: `Search` with `吴京的老婆`


["Xie Nan (谢楠) type: Chinese actor ‧ Wu Jing's wife.", 'Xie Nan (谢楠) born: 1983 (age 40 years), Hefei, China.', 'Xie Nan (谢楠) spouse: Wu Jing (m. 2013).', 'Xie Nan (谢楠) tv_shows: Healing Master, Snow Day.', 'Xie Nan (谢楠) nationality: Chinese.', 'Xie Nan (谢楠) grandchildren: Wu Zhi, Wu Ying.', '吴京老婆晒儿子做披萨，粉丝却把关注点放在脚上：遗传基因强大！ 丫丫健康美食 10.', '而作为最大的赢家吴京更是为了宣传电影，常常请别人吃饭，导致身材大走样，让不少人觉得战狼成了灰太狼的既视感！ 吴京最成功的不是他拍摄的电影，而是娶了一位好老婆，吴京在不出名的时候就将已是知名主持人的谢楠娶回了家，而得知吴京要拍电影，谢楠更是义无反顾的支持着吴京！ 也让吴京在战狼1的时候有 ...', '吴京有几任老婆- 百度 吴京\ue641目前只有一任老婆 ，她的名字是谢楠\ue641。 吴京和谢楠是在2014年结婚的，他们的两个孩子分别是吴所谓，出生于2014年，和吴虑，出生于2018年。 搜索结果中提到的其他女性，如樊亦敏和兰玉，并不是吴京的第一任或现任妻子，因此这部分信息是错误的。', '吴京还在婚礼上公开承认爱妻谢楠已怀有身孕，如今一家三口十分美满。而吴京在婚前因为受伤而影响活动，尽管身体稍有不适，但全程依然是甜蜜蜜的。', '#谢楠，83年11月6日出生于安徽省合肥，毕业于安徽财经大学，中国内地女主持人、演员，娱乐圈公认的“旺夫”女星，吴京背后的女人 · @ 高清影视混剪.', '吴京老婆谢楠二胎产子，无虑同学闪亮登场，网友：这手势很六哦！ 照片中，一家四口的拳头给人一种温馨感，而在细节上不得不说吴京谢楠夫妇做得十分 ...', '视频里两人夜里在街上漫步，吴京整个人都挂在老婆身上，像个人形挂件一样。 吴京和谢楠结婚已经9年了，这些年来，两人出

{'input': '谁是吴京的老婆？她现在的年龄是多少？',
 'output': '吴京的老婆是谢楠（Xie Nan）。她现在的年龄是40岁（出生于1983年）。'}